In [1]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_regression  # Veri kümesi oluşturmak için örnek
from sklearn.preprocessing import MinMaxScaler

In [29]:
X = data_USD[["Open", "High", "Low", "Day", "Month", "Year"]]  # Bağımsız değişkenler
y= data_USD["Close"]

y = np.ravel(y)

In [33]:
cols = ['Open', 'High', 'Low', 'Close']

for col in cols:
    # , → . çevir
    data_USD[col] = data_USD[col].astype(str).str.replace(',', '.', regex=False)
    # string → float
    data_USD[col] = pd.to_numeric(data_USD[col], errors='coerce')

# Eksik varsa at (isteğe bağlı)
data_USD = data_USD.dropna()

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
ext_reg = ExtraTreesRegressor(n_estimators=100, random_state=42)

In [40]:
X = data_USD[["Open", "High", "Low", "Day", "Month", "Year"]]
y = data_USD["Close"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score

ext_reg = ExtraTreesRegressor(n_estimators=100, random_state=42)
ext_reg.fit(X_train, y_train)

# Tahmin ve metrikler
train_predictions = ext_reg.predict(X_train)
test_predictions = ext_reg.predict(X_test)

print("Eğitim MSE:", mean_squared_error(y_train, train_predictions))
print("Test MSE:", mean_squared_error(y_test, test_predictions))
print("Eğitim R²:", r2_score(y_train, train_predictions))
print("Test R²:", r2_score(y_test, test_predictions))


Eğitim MSE: 1.0698469073887142e-28
Test MSE: 0.03314431053146155
Eğitim R²: 1.0
Test R²: 0.9991141639817188


Manual Search for Hyperparameters

In [41]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score
from itertools import product

# Hiperparametre listeleri
trees = [50, 100, 200]
max_depths = [None, 10, 20]
min_samples_splits = [2, 5, 10]
min_samples_leaves = [1, 2, 4]
max_features_opts = [None, "sqrt", "log2"]  # 'auto' yerine None kullanıldı

best_params = None
best_mse = float("inf")

# Farklı parametre kombinasyonlarını test et
for n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features in product(trees, max_depths, min_samples_splits, min_samples_leaves, max_features_opts):
    ext_reg = ExtraTreesRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    ext_reg.fit(X_train, y_train)
    test_predictions = ext_reg.predict(X_test)
    test_mse = mean_squared_error(y_test, test_predictions)
    
    if test_mse < best_mse:
        best_mse = test_mse
        best_params = {
            "n_estimators": n_estimators,
            "max_depth": max_depth,
            "min_samples_split": min_samples_split,
            "min_samples_leaf": min_samples_leaf,
            "max_features": max_features
        }

print("En iyi parametreler:", best_params)
print("En düşük Test MSE:", best_mse)

# En iyi parametrelerle modeli eğit
best_model = ExtraTreesRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)
train_predictions = best_model.predict(X_train)
test_predictions = best_model.predict(X_test)

# Son performans değerlendirmesi
train_mse = mean_squared_error(y_train, train_predictions)
test_mse = mean_squared_error(y_test, test_predictions)
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"Eğitim Verisi MSE: {train_mse}")
print(f"Test Verisi MSE: {test_mse}")
print(f"Eğitim Verisi R² Skoru: {train_r2}")
print(f"Test Verisi R² Skoru: {test_r2}")

En iyi parametreler: {'n_estimators': 50, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None}
En düşük Test MSE: 0.03148086865509779
Eğitim Verisi MSE: 0.0025685300233840628
Test Verisi MSE: 0.03148086865509779
Eğitim Verisi R² Skoru: 0.9999303372242279
Test Verisi R² Skoru: 0.9991586221920352


GRID SEARCH

In [42]:
# Grid Search ile hiperparametre optimizasyonu
grid_params = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"]
}

grid_search = GridSearchCV(ExtraTreesRegressor(random_state=42), grid_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print("Grid Search En İyi Parametreler:", grid_search.best_params_)
print("Grid Search En İyi MSE:", -grid_search.best_score_)
print("Grid Search En İyi R2:", grid_search.best_estimator_.score(X_test, y_test))

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Grid Search En İyi Parametreler: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Grid Search En İyi MSE: 0.024593394674114664
Grid Search En İyi R2: 0.9991099607337275


RANDOM SEARCH

In [43]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from scipy.stats import randint

# Parametre aralığını tanımlayın
param_dist = {
    'n_estimators': randint(50, 200),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'bootstrap': [True, False]
}

# ExtraTreesRegressor modelini oluşturun
ext_reg = ExtraTreesRegressor(random_state=42)

# RandomizedSearchCV'yi tanımlayın
random_search = RandomizedSearchCV(ext_reg, param_distributions=param_dist, 
                                   n_iter=100, cv=5, 
                                   verbose=2, random_state=42, n_jobs=-1)

# Modeli eğitin
random_search.fit(X_train, y_train)

# En iyi parametreleri ve skoru yazdırın
print(f"En İyi Parametreler: {random_search.best_params_}")
print(f"En İyi MSE: {-random_search.best_score_}")

# En iyi modeli kullanarak tahminler yapın
train_predictions = random_search.best_estimator_.predict(X_train)
test_predictions = random_search.best_estimator_.predict(X_test)

# MSE ve R² skorlarını hesaplayın
train_mse = mean_squared_error(y_train, train_predictions)
test_mse = mean_squared_error(y_test, test_predictions)

print(f"Eğitim Verisi MSE: {train_mse}")
print(f"Test Verisi MSE: {test_mse}")

train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"Eğitim Verisi R² Skoru: {train_r2}")
print(f"Test Verisi R² Skoru: {test_r2}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
145 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
140 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\ProgramData\anaconda3\Lib\site-packag

En İyi Parametreler: {'bootstrap': True, 'max_depth': 46, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 123}
En İyi MSE: -0.9991658494902602
Eğitim Verisi MSE: 0.004175595538487262
Test Verisi MSE: 0.03884133449732873
Eğitim Verisi R² Skoru: 0.9998867509536332
Test Verisi R² Skoru: 0.9989619016795301
